In [348]:
#import libraries
import requests
import pandas as pd
from datetime import datetime
import os
import html

In [350]:
#Add trafic2023_ratio or raw somewhere?

In [448]:
#download dataset
df_lines = pd.read_csv("./Online_Data/referentiel-des-lignes.csv", sep=';')
stops_data = pd.read_csv("./Online_Data/arrets.csv", sep=';')

trafic2023_ratio = pd.read_csv("./Online_Data/validations-1er-semestre.csv", sep=';')
trafic2023_raw = pd.read_csv("./Online_Data/validations-reseau.csv", sep=";")

hourly_weather = pd.read_csv("./Online_Data/Weather/hourly_weather.csv")
minutely_15_weather = pd.read_csv("./Online_Data/Weather/minutely_15_weather.csv")

delays_metro = pd.read_csv("./Collected_Data/metro_delays.csv")
delays_rer = pd.read_csv("./Collected_Data/rail_delays.csv")
onTime_metro = pd.read_csv("./Collected_Data/metro_onTime.csv")
onTime_rer = pd.read_csv("./Collected_Data/rail_onTime.csv")

metro_incident = pd.read_csv("./Collected_Data/metro_line_reports.csv")
rer_incident = pd.read_csv("./Collected_Data/rer_line_reports.csv")
#gas = pd.read_csv("./Online_Data/prix-des-carburants-en-france-flux-instantane-v2.csv", sep=";") #needs cleaning/filtering

In [449]:
#create filter
stops_filter = [22086, 463013, 22136, 462993, 21964, 462969, 22125, 463113, 41295, 473921, 473993, 41354, 474060, 474061]
name_filter = ["CH.D.G.ETOILE", "CHATELET", "SAINT-LAZARE","ST-GERM.D.PRES", "BLANCHE","AVENUE DU PRESIDENT KENNEDY","BUNO GIRONVILLE","MASSY PALAISEAU"]
lines_filter = ["C01371", "C01372", "C01374" ," C01382", "C01742", "C01743", "C01727", "C0172"]

In [450]:
#clean dataset
line_refs = df_lines[(~df_lines['TransportSubmode'].isin(['suburbanRailway', 'regionalRail', 'railShuttle']))]
line_refs = line_refs[['ID_Line', 'TransportMode', 'Name_Line']]
line_refs = line_refs.sort_values(by='ID_Line')

stops_data = stops_data[stops_data['ArRType'].isin(['metro', 'rail'])]
stops_data = stops_data.sort_values(by=['ArRType', 'ArRId'])
stops_data = stops_data[['ArRId', 'ArRName', 'ArRType', 'ArRTown']]

def cleaning_message(df):
    df['ref'] = df['ref'].str.replace('stop_point:IDFM:', '', regex=False)
    df['ref'] = df['ref'].str.replace('line:IDFM:', '', regex=False)
    df['ref'] = df['ref'].str.replace('stop_area:IDFM:', '', regex=False)
    df['message_text'] = df['message_text'].str.replace('<p>', '', regex=False)
    df['message_text'] = df['message_text'].str.replace('<br>', '', regex=False) 
    df = df[df["channel_name"].isin(["moteur"])]

cleaning_message(metro_incident)
cleaning_message(rer_incident)

metro_incident['message_text'] = metro_incident['message_text'].str.split('</p>').str[0]
metro_incident.drop_duplicates(subset=["disruption_id"], inplace=True)
metro_incident.drop(['tags', 'category', 'updated_at', 'channel_name'], axis=1, inplace=True) 

rer_incident['message_text'] = rer_incident['message_text'].str.replace('</p>', '', regex=False)
rer_incident['message_text'] = rer_incident['message_text'].apply(html.unescape)
rer_incident.drop_duplicates(subset=["disruption_id"], inplace=True)
rer_incident.drop(['tags', 'category', 'updated_at', 'channel_name'], axis=1, inplace=True) 

delays_metro.drop(['scheduled_arrival','scheduled_departure','arrival_difference','departure_difference'], axis=1, inplace=True)
delays_metro['line_ref'] = delays_metro['line_ref'].str.replace('STIF:Line::', '', regex=False).str.rstrip(':')
onTime_metro.drop(['scheduled_arrival','scheduled_departure','arrival_difference','departure_difference'], axis=1, inplace=True)
onTime_metro['line_ref'] = delays_metro['line_ref'].str.replace('STIF:Line::', '', regex=False).str.rstrip(':')

mapping = dict(zip(name_filter, stops_filter))
trafic2023_ratio['LIBELLE_ARRET_REA'] = trafic2023_ratio['LIBELLE_ARRET'].replace(mapping)
trafic2023_ratio.drop(["lda"],axis=1, inplace=True)

trafic2023_raw['LIBELLE_ARRET_REA'] = trafic2023_raw['LIBELLE_ARRET'].replace(mapping)
trafic2023_raw.drop(["lda"],axis=1, inplace=True)

In [451]:
#filter dataset
stops_data = stops_data[stops_data['ArRId'].isin(stops_filter)]

metro_incident = metro_incident[
    metro_incident['ref'].isin(map(str, stops_filter + lines_filter))
]

rer_incident = rer_incident[
    rer_incident['ref'].isin(map(str, stops_filter + lines_filter))
]

trafic2023_ratio = trafic2023_ratio[trafic2023_ratio['LIBELLE_ARRET_REA'].isin(stops_filter)]

trafic2023_raw = trafic2023_raw[trafic2023_raw['LIBELLE_ARRET_REA'].isin(stops_filter)]

In [452]:
#Merge datasets
merged_metro = pd.concat([delays_metro, onTime_metro], ignore_index=True)
merged_metro = pd.merge(merged_metro, stops_data, left_on='stop_reference', right_on='ArRId')
merged_metro.drop(['ArRId', 'ArRName', 'ArRType', 'transport_mode', 'recorded_at_time'], axis=1, inplace=True) 
merged_metro = merged_metro.sort_values(by='real_arrival')

merged_rer = pd.concat([delays_rer, onTime_rer], ignore_index=True)
merged_rer = pd.merge(merged_rer, stops_data, left_on='stop_reference', right_on='ArRId')
merged_rer.drop(['ArRId', 'ArRName', 'ArRType', 'transport_mode', 'recorded_at_time'],  axis=1, inplace=True)
merged_rer = merged_rer.sort_values(by='scheduled_arrival')

def date_format(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['real_arrival'] = pd.to_datetime(df['real_arrival']).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['real_arrival'] = pd.to_datetime(df['real_arrival'])
    df['real_departure'] = pd.to_datetime(df['real_departure']).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['real_departure'] = pd.to_datetime(df['real_departure'])

def format_rer(df):
    df['scheduled_arrival'] = pd.to_datetime(df['scheduled_arrival']).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['scheduled_arrival'] = pd.to_datetime(df['scheduled_arrival'])
    df['scheduled_departure'] = pd.to_datetime(df['scheduled_departure']).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['scheduled_departure'] = pd.to_datetime(df['scheduled_departure'])
    df.loc[(df['arrival_difference'] >= 5) | (df['departure_difference'] >= 5), 'departure_status'] = 'delayed'

date_format(merged_metro)
date_format(merged_rer)
format_rer(merged_rer)

In [453]:
#Clean Gas Price data
def clean_gas_data(file_path, fuel_type):
    # Load CSV file
    df = pd.read_csv(file_path, skiprows=3, sep=';')  # Skip the first 3 metadata rows
    df.rename(columns={df.columns[1]: f'{fuel_type}'}, inplace=True)
    df = df.iloc[:, :-1]
    return df
    
gas_95_df = clean_gas_data("./Online_Data/Gas/octane_95.csv", "95")
gas_98_df = clean_gas_data("./Online_Data/Gas/octane_98.csv", "98")
gas_e10_df = clean_gas_data("./Online_Data/Gas/95-E10.csv", "E10")
gazole_df = clean_gas_data("./Online_Data/Gas/gazole.csv", "gazole")

merged_rer['Période'] = pd.to_datetime(merged_rer['timestamp']).dt.strftime('%Y-%m')
merged_rer = merged_rer.merge(gas_95_df, on='Période', how='left')
merged_rer = merged_rer.merge(gas_98_df, on='Période', how='left')
merged_rer = merged_rer.merge(gas_e10_df, on='Période', how='left')
merged_rer = merged_rer.merge(gazole_df, on='Période', how='left')

merged_rer.drop(columns=['Période'], inplace=True)

In [463]:
hourly_weather['date'] = pd.to_datetime(hourly_weather['date'])
minutely_15_weather['date'] = pd.to_datetime(minutely_15_weather['date'])

merged_metro = pd.merge_asof(merged_metro, 
                   minutely_15_weather, 
                   left_on='real_arrival', 
                   right_on='date', 
                   direction='nearest')

merged_rer = pd.merge_asof(merged_rer, 
                   minutely_15_weather, 
                   left_on='scheduled_arrival', 
                   right_on='date', 
                   direction='nearest')

,timestamp,stop_reference,stop_name,line_ref,destination_name,departure_status,scheduled_arrival,real_arrival,scheduled_departure,real_departure,...,gazole,date,temperature_2m,precipitation,rain,snowfall,wind_speed_10m,wind_gusts_10m,visibility,is_day
0,2024-10-30 05:27:09,473921,Châtelet - Les Halles,STIF:Line::C01742:,Saint-Germain-en-Laye,delayed,2024-10-30 04:30:38,2024-10-30 04:36:50,2024-10-30 04:30:38,2024-10-30 04:36:50,...,1.63,2024-10-30 04:30:00,11.10,0.0,0.0,0.0,1.48,5.04,3380.0,False
1,2024-10-30 05:23:03,41295,Avenue du Président Kennedy,STIF:Line::C01727:,Gare de Pontoise,delayed,2024-10-30 05:05:00,2024-10-30 05:11:32,2024-10-30 05:06:00,2024-10-30 05:12:32,...,1.63,2024-10-30 05:00:00,11.05,0.0,0.0,0.0,1.80,4.32,3220.0,False
2,2024-10-30 06:23:36,41295,Avenue du Président Kennedy,STIF:Line::C01727:,Gare de Pontoise,delayed,2024-10-30 05:05:00,2024-10-30 05:10:30,2024-10-30 05:06:00,2024-10-30 05:11:30,...,1.63,2024-10-30 05:00:00,11.05,0.0,0.0,0.0,1.80,4.32,3220.0,False
3,2024-10-30 06:23:36,41295,Avenue du Président Kennedy,STIF:Line::C01727:,Gare de Montigny Beauchamp,delayed,2024-10-30 05:21:20,2024-10-30 05:33:21,2024-10-30 05:22:20,2024-10-30 05:34:21,...,1.63,2024-10-30 05:15:00,11.10,0.0,0.0,0.0,2.19,4.68,2960.0,False
4,2024-10-30 07:24:40,41295,Avenue du Président Kennedy,STIF:Line::C01727:,Gare de Montigny Beauchamp,delayed,2024-10-30 06:03:20,2024-10-30 06:12:30,2024-10-30 06:04:20,2024-10-30 06:13:30,...,1.63,2024-10-30 06:00:00,11.25,0.0,0.0,0.0,4.10,5.40,2180.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818,2024-12-19 18:14:33,41354,Gare de Buno Gironville,STIF:Line::C01728:,Malesherbes,onTime,2024-12-19 19:46:40,2024-12-19 19:46:40,2024-12-19 19:47:10,2024-12-19 19:47:10,...,NaN,2024-11-27 23:45:00,8.55,0.0,0.0,0.0,5.45,17.28,24340.0,False
819,2024-12-19 18:14:12,41295,Avenue du Président Kennedy,STIF:Line::C01727:,Gare de Montigny Beauchamp,onTime,2024-12-19 19:51:20,2024-12-19 19:51:20,2024-12-19 19:52:20,2024-12-19 19:52:20,...,NaN,2024-11-27 23:45:00,8.55,0.0,0.0,0.0,5.45,17.28,24340.0,False
820,2024-12-19 18:14:12,41295,Avenue du Président Kennedy,STIF:Line::C01727:,Massy-Palaiseau,onTime,2024-12-19 19:53:10,2024-12-19 19:53:10,2024-12-19 19:54:10,2024-12-19 19:54:10,...,NaN,2024-11-27 23:45:00,8.55,0.0,0.0,0.0,5.45,17.28,24340.0,False
821,2024-12-19 18:14:12,41295,Avenue du Président Kennedy,STIF:Line::C01727:,Gare de Pontoise,onTime,2024-12-19 20:06:20,2024-12-19 20:06:20,2024-12-19 20:07:20,2024-12-19 20:07:20,...,NaN,2024-11-27 23:45:00,8.55,0.0,0.0,0.0,5.45,17.28,24340.0,False


In [465]:
#split by stop
CDG = merged_metro[merged_metro['stop_name'].isin(['Charles de Gaulle-Etoile'])]
SGP = merged_metro[merged_metro['stop_name'].isin(['Saint-Germain des Prés'])]
BL = merged_metro[merged_metro['stop_name'].isin(['Blanche'])]
SL = merged_metro[merged_metro['stop_name'].isin(['Saint-Lazare'])]
APK = merged_rer[merged_rer['stop_name'].isin(['Avenue du Président Kennedy'])]
CLH = merged_rer[merged_rer['stop_name'].isin(['Châtelet - Les Halles'])]
GBG = merged_rer[merged_rer['stop_name'].isin(['Gare de Buno Gironville'])]
MP = merged_rer[merged_rer['stop_name'].isin(['Massy - Palaiseau'])]